# Part-based R-CNNs for Fine-grained Category Detection  

## Abstract  

语义部分本地化可以通过明确隔离与特定对象部分相关的细微外观差异来促进细粒度分类。 已经提出了用于姿态规范化表示的方法，但由于对象检测的困难，通常在测试时间假定边界框注释。 我们提出了一个精细粒度分类模型，通过利用由下至上区域提议计算的深度卷积特性来克服这些限制。 我们的方法学习全对象和部分检测器，在它们之间强制学习几何约束，并从姿态归一化表示预测细粒度类别。 对CaltechUCSD鸟类数据集的实验证实，我们的方法在端到端评估中胜过了最先进的细粒度分类方法，而无需在测试时间使用边界框。  

## 1. Introduction  

由于相关类别中某些部分外观的细微差异，视觉细粒度分类问题可能非常具有挑战性。 与基本级别识别相比，细粒度分类旨在区分不同品种或物种或产品模型，并且通常需要区分，必须以物体姿态为条件进行可靠识别。 面部识别是细粒度识别的经典案例，值得注意的是最好的面部识别方法联合发现面部标志并从这些位置提取特征。  

因此，本地化对象中的部件对于建立对象实例之间的对应关系以及折扣对象姿势变化和相机视图位置至关重要。 以前的工作已经调查了基于部件的方法来解决这个问题[7,16,30,44,47,22]。 许多姿态归一化表示的瓶颈确实是精确的零件定位。 之前已经使用了Poselet [8]和DPM [17]方法来取得适度成功的部分定位; 只有在测试时间给出已知的边界框时，方法通常才报告足够的部分定位[11,20,36,37,43]。 通过开发一种新颖的深部件检测方案，我们提出了一种端到端的细粒度分类系统，该系统在测试时不需要对象边界框的知识，并且可以实现与先前报道的方法相媲美的性能，这些方法需要测试时间的地面真实边界框 过滤误报检测  

卷积网络最近在ImageNet挑战[23]上的成功激发了将深度卷积特征应用于相关图像分类[14]和检测任务[21]的进一步工作。 在[21]中，Girshick等人。 通过将[27]的CNN应用于一组自下而上的候选区域提议[41]，实现了对象检测的突破性性能，与以前的最佳方法相比，PASCAL检测性能提高了30％以上。 独立地，OverFeat [38]提出使用CNN进行本地化以回归到对象位置。 但是，利用深度卷积特征的进展并不局限于基本的目标检测。 在许多应用中，如细粒度识别，属性识别，姿态估计等，合理的预测需要准确的零件定位。  

特征学习已被用于细粒度识别和属性估计，但仅限于本地化的工程特征。 DPD-DeCAF [48]使用DeCAF [14]作为特征描述符，但依靠基于HOG的DPM [17]进行零件定位。 PANDA [49]学习了特定部分深度卷积网络，其位置取决于基于HOG的poselet模型。 这些模型缺乏R-CNN的强度和检测鲁棒性[21]。 在这项工作中，我们探索了一种统一的方法，它使用相同的深度卷积表示进行检测以及部分描述。  

我们推测，自下而上区域提议方法取得的进展，如选择性搜索[41]，除了整个对象外，还可以使小部分的本地化受益。 正如我们后面显示的那样，Caltech-UCSD鸟类数据集使用选择性搜索建议的平均召回率为95％。  

在本文中，我们提出了一个部分定位模型，它通过利用计算自下而上区域提议的深度卷积特征来克服先前的细粒度识别系统的局限性。 我们的方法学习零件外观模型并强化零件之间的几何约束。 我们的方法概述如图1所示。我们已经研究了不同的几何约束，包括在语义外观空间中以最近邻居为条件的关节部位位置的非参数模型。 我们提供最新的结果评估我们的方法在广泛使用的细粒度基准Caltech-UCSD鸟类数据集[42]。  

## 2.  Related work  

### 2.1 Part-based models for detection and pose localization  

以前的工作已经提出了对对象部分外观和位置的显式建模，以便更精确地识别和定位。 从图像结构[18,19]开始，继续通过poselets [8]和相关工作，许多方法已经共同定位了一组几何相关的部分。 可变形零件模型（DPM）[17]，直到最近还有最先进的PASCAL物体检测方法，它们在相对于整个物体边界框固定的位置上模拟具有额外学习过滤器的零件，从而允许零件从 这与学习变形成本锚。 “强”的DPM [3]将这种方法用于强烈监督的设置，其中部分位置在训练时注释。 这些方法的局限性在于它们使用弱特征（通常是HOG [12]）。  

### 2.2 Fine-grained categorization  

最近，大量的计算机视觉研究集中在许多领域的细粒度分类问题，如动物品种或物种[16,26,31,33,37,46]，植物物种[1,2,5 ，34,35,39]和人造物体[32,40]。  

有几种方法基于检测和提取物体某些部分的特征。法雷尔等人。 [16]提出了一种使用poselets的姿态归一化表示法[8]。可变形零件模型[17]在[37,48]中用于零件定位。基于定位脸部基准标志的工作[6]，Liu等人[31]提出了一种基于样本的几何方法来检测狗脸并从关键点提取高度局部化的图像特征以区分狗的品种。此外，Berg等人[7]通过学习每对关键点的描述符，学习了一组高度区分的中间特征。此外，在[30]中，作者通过强化姿态和子类一致性来扩展[6]的非参数样本法。姚等人。 [45]和杨等人。 [44]研究了模板匹配方法以降低滑动窗口方法的成本。 G¨oring等人最近的工作。 [22]将部分注释从具有类似全局形状的对象转换为非参数部分检测。但是，所有这些基于零件的方法在测试时都需要地面真值边界框来进行零件定位或关键点预测。  

人在回路中的方法[9,13,15]要求人们命名对象的属性，点击某些部分或标记最具区分性的区域以提高分类的准确性。基于细分的方法对于细粒度识别也是非常有效的。诸如[11,20,36,37,43]等方法使用区域级提示来推断前景分割掩模并丢弃背景中的噪声视觉信息。柴等。 [10]表明联合学习部分定位和前景分割可以有益于细粒度分类。与大多数先前的基于零件的方法类似，这些努力需要地面真值边界框来初始化分割种子。相比之下，我们的工作目标是在地面真值边界框的测试时间内不知道端到端的细粒度分类。我们的部分检测器使用基于自底向上区域提议的卷积特征，以及学习的非参数几何约束来更精确地定位对象部分，从而实现强大的细粒度分类。  

### 2.3  Convolutional networks  

近年来，卷积神经网络（CNN）已被纳入各种领域的许多视觉识别系统中。 这些模型的强度至少有一部分在于它们能够从原始数据输入（例如图像像素）中学习辨别特征，这与作为初始预处理步骤计算图像上的手工设计特征的更传统的物体识别流水线相反。 LeCun及其同事最初将这些模型应用于数字识别[28]和OCR [29]，后来应用于通用对象识别任务[24]，CNN得到了推广。 随着大型标签图像数据库[23]和GPU实现的引入，这些网络已经成为通用对象分类[27]中最准确的方法，用于高效地执行大型CNN中的学习和推理所需的大规模并行计算。  

最近，通用对象检测方法已经开始利用深度CNN，并且胜过基于传统特性的任何竞争方法。 OverFeat [38]使用CNN在粗滑动窗口检测框架中回归到对象位置。 对我们工作特别有启发的是R-CNN方法[21]，该方法利用区域提案框架中CNN的深层特征，在PASCAL VOC数据集上实现前所未有的目标检测结果。 我们的方法通过将R-CNN应用于除了整个对象之外的模型对象部分来推广R-CNN，我们的实证结果将证明这对精确的细粒度识别是至关重要的。  

## 3.  Part-based R-CNNs  

虽然[21]证明了R-CNN方法在通用物体检测任务（PASCAL VOC）上的有效性，但没有探索这种方法在同时定位和细粒度识别中的应用。 因为我们的工作是在这个机制中运作的，所以我们扩展了R-CNN来检测物体并将它们的零件定位在几何之前。 利用关于感兴趣对象的单个语义部分的位置的假设（例如，用于动物类的头部位置），对于倾向于出现在相对于这些粗略固定的位置中的微妙外观差异进行建模变得合理 部分。  

在R-CNN方法中，对于特定对象类别，具有CNN特征描述符φ（x）的候选检测x被赋予w | 0φ（x），其中w0是对象类别的SVM权重的学习向量。 在我们的方法中，我们假设一个强有力的监督设置（例如，[3]），其中在训练时间，我们不仅为整个对象设定了地面真值边界框注释，而且为一组固定的语义部分{p1，p2，.. 。，pn}。  

在给定这些部分注释的情况下，在训练时，所有对象及其每个部分都被视为独立的对象类别：我们在区域提议中提取的特征描述符上训练一对一线性支持向量机，其中≥0.7的区域与地面真值对象重叠或部分边界框被标记为该物体或部分的正数，并且与任何地面真值区域重叠≤0.3的区域被标记为负数。因此，对于单个对象类别，我们分别为部分{p1，p2，...，pn}学习全对象（“根”）SVM权重w0和部分SVM权重{w1，w2，...，wn}。在测试时间，对于每个地区提案窗口，我们计算来自所有根和部分SVM的分数。当然，这些分数并不包含任何有关物体及其部件几何约束的知识;例如，在没有任何额外的限制的情况下，鸟头探测器可以在鸟探测器发射的区域之外发射。因此，我们的最终联合对象和部分假设是使用下面几节中详述的几何打分函数来计算的，它强化了姿势预测与训练时观察姿势的统计数据一致的直观理想属性。